# CSC421 Assignment 3 - Part II Naive Bayes Classification (5 points) #
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 13 Quantifying Uncertainty**and **Chapter 20 - Statistical Learning Method** from the book *Artificial Intelligence: A Modern Approach.* This part does NOT rely on the provided code so you can complete it just using basic Python. 

```
Misunderstanding of probability may be the greatest of all impediments
to scientific literacy.

Gould, Stephen Jay
```



# Introduction 


Text categorization is the task of assigning a given document to one of a fixed set of categories, on the basis of text it contains. Naive Bayes models are often used for this task. In these models, the query variable is
the document category, and the effect variables are the presence/absence
of each word in the language; the assumption is that words occur independently in documents within a given category (condititional independence), with frequencies determined by document category. Download the following file: http://www.cs.cornell.edu/People/pabo/movie-review-data/review_polarity.tar.gz containing a dataset that has been used for text mining consisting of movie reviews classified into negative and positive. You
will see that there are two folders for the positivie and negative category and they each contain multiple text files with the reviews. You can find more information about the dataset at: 
http://www.cs.cornell.edu/People/pabo/movie-review-data/


Our goal will be to build a simple Naive Bayes classifier for this dataset. More complicated approaches using term frequency and inverse document frequency weighting and many more words are possible but the basic concepts
are the same. The goal is to understand the whole process so DO NOT use existing machine learning packages but rather build the classifier from scratch.

Our feature vector representation for each text file will be simply a binary vector that shows which of the following words are present in the text file: Awful Bad Boring Dull Effective Enjoyable Great Hilarious. For example the text file cv996 11592.txt would be represented as (0, 0, 0, 0, 1, 0, 1, 0) because it contains Effective and Great but none of the other words.

# Question 2A (Minimum) CSC421 -  (1 point, CSC581C - 0 points) 

Write code that parses the text files and calculates the probabilities for
each dictionary word given the review polarity

In [33]:
import os
import numpy as np
from tabulate import tabulate
from random import random

words = ['awful', 'bad', 'boring', 'dull', 'effective', 'enjoyable', 'great', 'hilarious']

In [10]:
def parse_prob(polarity):
    prob = np.zeros(8)
    for s, d, files in os.walk('txt_sentoken' + os.sep + polarity):
        for file in files:
            with open('txt_sentoken' + os.sep + polarity + os.sep + file) as f:
                review = f.read().lower()
                prob += np.array([1 if word in review else 0 for word in words])
        return [x / len(files) for x in prob]

print(tabulate([
    ['pos'] + parse_prob('pos'),
    ['neg'] + parse_prob('neg')],
    headers= ['polarity'] + words
))

polarity      awful    bad    boring    dull    effective    enjoyable    great    hilarious
----------  -------  -----  --------  ------  -----------  -----------  -------  -----------
pos           0.034  0.28      0.054   0.025        0.154        0.096    0.485        0.132
neg           0.122  0.545     0.175   0.101        0.086        0.054    0.32         0.059


# Question 2B (Minimum) (CSC421 - 1 point, CSC581C - 0 point) 


Explain how the probability estimates for each dictionary word given the review polarity can be combined to form a Naive Bayes classifier. You can look up Bernoulli Bayes model for this simple model where only presence/absence of a word is modeled.

Your answer should be a description of the process with equations and a specific example as markdown text NOT python code. You will write the code in the next questinon. 

### Bayes’ Theorem
$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

<h6>Positive Review Proability</h6> 

$P(POS|review) = a * P(aweful|POS)P(bad|POS)...P(hilarious|POS)P(POS)$

<h6>Negative Review Proability</h6> 

$P(NEG|review) = a * P(aweful|NEG)P(bad|NEG)...P(hilarious|NEG)P(NEG)$

<h6>Assumption</h6>

$P(NEG) = P(POS) = 1000 / 2000 = 0.5$

<h6>Example</h6>

review = "bad grandpa was an awfully hilarious movie. johnny knoxville was enjoyable as always, there wasn't a dull or boring moment. 5/7 great movie"

feature vector for the above review = [1, 1, 1, 1, 0, 1, 1, 1]

P(POS|review) = a * (0.034 * 0.28 * 0.054 * 0.025 * (1 - 0.154) * 0.096 * 0.485 * 0.132) * 0.5
              = a * 3.341165490432001e-08
               
P(NEG|review) = a * (0.122 * 0.545 * 0.175 * 0.101 * (1 - 0.086) * 0.054 * 0.32 * 0.059) * 0.5
              = a * 5.475549447748801e-07
              
3.341165490432001e-08 + 5.475549447748801e-07 = 5.809665996792001e-07


P(POS|review) = 3.341165490432001e-08 / 5.809665996792001e-07 = 0.05751045743898076 ~ 6%

P(NEG|review) = 5.475549447748801e-07 / 5.809665996792001e-07 = 0.9424895425610192 ~ 94%

# Question 2C (Expected) 1 point 

Write Python code for classifying a particular test instance (in our case movie review) following a Bernolli Bayes approach. Your code should calculate the likelihood the review is positive given the correspondng conditional probabilities for each dictionary word as well as the likelihood the review is negative given the corresponding conditional probabilities for each dictionary word. Check that your code works by providing a few example cases of prediction. Your code should be written from "scratch" and only use numpy/scipy but not machine learning libraries like scikit-learn or tensorflow. 


In [19]:
def classify(review):
    pos = [0.034, 0.28, 0.054, 0.025, 0.154, 0.096, 0.485, 0.132]
    neg = [0.122, 0.545, 0.175, 0.101, 0.086, 0.054, 0.32, 0.059]
    
    review = review.lower()
    vector = np.array([1 if word in review else 0 for word in words])
    
    p, n = 0.5, 0.5 # P(POS), P(NEG) 
    for word, p_pos, p_neg in zip(vector, pos, neg):
        p *= p_pos if word else (1 - p_pos)
        n *= p_neg if word else (1 - p_neg)
    
    return p / (p + n), n / (p + n)

review_1 = "bad grandpa was an awfully hilarious movie. johnny knoxville was enjoyable as always, there wasn't a dull or boring moment. 5/7 great movie"
print(f'review 1: {classify(review_1)}')   

review_2 = "avatar was bad, awful, and an effective waste of $20, the only redeeming quaility was the great 3d effects"
print(f'review 2: {classify(review_2)}')

review_3 = "toy story 4 was a fantastic movie. it was a great way to wrap up the story. the ending was enjoyable yet sad. the toys antics were hilarious as always"
print(f'review 3: {classify(review_3)}')

review 1: (0.05751045743898076, 0.9424895425610192)
review 2: (0.2987254534254119, 0.7012745465745882)
review 3: (0.9235529619927509, 0.07644703800724907)


# QUESTION 2D (Expected ) 1 point

Calculate the classification accuracy and confusion matrix that you would obtain using the whole data set for both training and testing. Do not use machine learning libraries like scikit-learn or tensorflow for this only the basic numpy/scipy stuff. 

In [31]:
def classification_accuracy():
    matrix = np.zeros((2, 2))
    for s, d, files in os.walk('txt_sentoken'):
        for file in files:
            with open(s + os.sep + file) as f:
                pos, neg = classify(f.read().lower())
                if 'pos' in s:
                    if pos > neg:
                        matrix[0][0] += 1
                    else:
                        matrix[0][1] += 1
                else:
                    if neg > pos:
                        matrix[1][0] += 1
                    else:
                        matrix[1][1] += 1
                        
    accuracy = (matrix[0][0] + matrix[1][0]) / sum(sum(matrix))                
    return accuracy, matrix
                 
a, m = classification_accuracy()

print(f'classification accuracy: {a}')
print('confusion matrix:')
print(m.tolist()[0])
print(m.tolist()[1])

classification accuracy: 0.6705
confusion matrix:
[720.0, 280.0]
[621.0, 379.0]


# QUESTION 2E (Advanced) 1 point 

One can consider the Naive Bayes classifier a generative model that can generate binary feature vectors using the associated probabilities from the training data. The idea is similar to how we do direct sampling in
Bayesian Networks and depends on generating random number from a discrete distribution. Describe how you would generate random movie reviews consisting solely of the words from the dictionary using your model. Show 5 examples of randomly generated positive reviews and 5 examples of randomly generated negative reviews. Each example should consists of a subset of the words in the dictionary. Hint: use probabilities to generate both the presence and absence of a word

In [63]:
# 1. generate an array of probabilities for each word
# 2. if the probability of the word is greater than the random probability 
#        3a. include the word
#    else
#        3b. exclude the word
#
# Note:
#       this only generates an array of words likely to be in the review
#
#       the implimentation below slightly changes the described implimentation using list comprehension

def generate_review(polarity):
    prob = None
    if polarity == 'pos':
        prob = [0.034, 0.28, 0.054, 0.025, 0.154, 0.096, 0.485, 0.132]
    else:
        prob = [0.122, 0.545, 0.175, 0.101, 0.086, 0.054, 0.32, 0.059]
    
    v = [1 if random() < p else 0 for p in prob]

    review = []
    for result, word in zip(v, words):
        if result:
            review.append(word)
    
    return review

pos = [generate_review('pos') for i in range(5)]
neg = [generate_review('neg') for i in range(5)]


print('positive reviews:')
print(pos)
print()
print('negative reviews:')
print(neg)


positive reviews:
[['awful', 'great'], [], ['enjoyable', 'great'], ['bad', 'effective', 'great'], ['awful']]

negative reviews:
[['bad', 'great'], ['bad', 'boring'], ['bad'], ['bad', 'hilarious'], ['bad', 'great']]


# QUESTION 2F (ADVANCED) (CSC421 - 0 points, CSC581C - 2 points)

Check the associated README file and see what convention is used for the 10-fold cross-validation. Calculate the classification accuracy and confusion matrix using the recommended 10-fold cross-validation. Again do NOT use 
ML libraries such as scikit-learn or tensorflow and just use numpy/scipy. 

In [50]:
# YOUR CODE GOES HERE 